# Collect data using API

## Plan:
1. Collect ids for all available car ads
2. Get info about each car by id

In [1]:
import requests
import time
import random
import json
import re
import sys

In [2]:
api_key = ''#put your API key here
get_ids_str = 'https://developers.ria.com/auto/search?api_key=' + api_key + '&category_id=1&countpage=100&currency=1&' \
+ 'abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000' + \
'&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3' + \
'&custom=1&page='

get_car_str = 'https://developers.ria.com/auto/info?api_key=' + api_key + '&auto_id='

## Step 0. Load already availible ids and last page_id

In [3]:
retrieved_ids = open("data/already_retrieved.txt", "r")

In [4]:
ids_str = retrieved_ids.read()

In [5]:
len(ids_str.splitlines())

0

In [6]:
already_retreived = set(ids_str.splitlines())
retrieved_ids.close()

In [7]:
last_page_id = open('data/last_page.txt', 'r')
ids_str = last_page_id.read()
ids_str = ids_str.splitlines()
page = 0
if len(ids_str) != 0:
    page = int(ids_str[-1])
last_page_id.close()

In [8]:
page

0

In [9]:
len(already_retreived)

0

## Step 1. get ids of availible cars

In [10]:
retrieved_ids_file = open("data/already_retrieved.txt", "a")
last_page_file = open('data/last_page.txt', 'a')

try:

    while page < 5: #use max_page here for the whole process. +5 is for demo. and be patient ;)

        getstr = get_ids_str + str(page)
        print(getstr)
        resp = requests.get(getstr)

        print('request №' + str(page) + ' was sent')
        print(resp.status_code)

        if resp.status_code != 200:
            resp.close()
            break
        else:
            ids = resp.json()['result']['search_result']['ids']
            for ido in ids:
                if not (ido in already_retreived):
                    retrieved_ids_file.write(ido)
                    retrieved_ids_file.write('\n')
                    already_retreived.add(ido)
            resp.close()
        page += 1
        time.sleep(0.3)
finally:
    last_page_file.write(str(page))
    last_page_file.write('\n')
    last_page_file.flush()
    last_page_file.close()
    retrieved_ids_file.flush()
    retrieved_ids_file.close()

https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3&custom=1&page=0
request №0 was sent
200
https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]=2&type[2]=5&type[3]=6&type[4]=3&custom=1&page=1
request №1 was sent
200
https://developers.ria.com/auto/search?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&category_id=1&countpage=100&currency=1&abroad=2&sellerType=1&damage=0&under_credit=0&confiscated_car=0&price_ot=100&price_do=100000&engineVolumeFrom=0.1&engineVolumeTo=10&type[0]=1&type[1]=2&type[2]=5&type[3]=6&typ

In [11]:
page

5

## Step 2. Get info about cars

### Write functions for processing and writing data

* **why random id?**
* **Should we write raw info?**

In [12]:
def get_random_id_and_del_from_list(ids_list):
    if(len(ids_list) != 0):
        i = random.randint(0, len(ids_list)-1)
        print(i)
        ido = ids_list[i]
        del ids_list[i]
        return ido
    else:
        return _

def write_raw_info(ido, json_info, path):
    raw_info_file = open(path+ido+'.json', "w", encoding='utf-8')
    json.dump(json_info,raw_info_file, ensure_ascii=False)
    raw_info_file.close()
    return _
            
    
def add_info_to_csv(ido, dict_info, file):
    USD = str(dict_info['USD'])
    markName = dict_info['markName']
    
    modelName = dict_info['modelName']
    
    try:
        markNameEng = dict_info['markNameEng']
    except:
        markNameEng = dict_info['markName'].lower().replace(' ', '-')
    
    try:
        modelNameEng = dict_info['modelNameEng']
    except:
        modelNameEng = dict_info['modelName'].lower().replace(' ', '-')
   
    
    try:
        raceInt = str(dict_info['autoData']['raceInt'])
    except:
        raceInt = re.search('[0-9]+', dict_info['autoData']['fuelName'])[0]
    
    year = str(dict_info['autoData']['year'])
    
    subCategoryNameEng = dict_info['autoData']['subCategoryNameEng']
    
    fuelName = re.search('[0-9.]+', dict_info['autoData']['fuelName'])[0]
    #file.write(dict_info['autoData']['fuelName'].split(', ')[1].split(' ')[0])
    
    fuelNameEng = dict_info['autoData']['fuelNameEng']
    
    gearboxName = dict_info['autoData']['gearboxName']
    
    
    custom = str(dict_info['autoData']['custom'])
    
    regionNameEng = dict_info['stateData']['regionNameEng']
    
    locationCityName = dict_info['locationCityName']
    
    count = str(dict_info['photoData']['count'])
    
    userId = str(dict_info['userId'])
    
    phone = dict_info['userPhoneData']['phone']
    
    
    
    file.write(ido)
    file.write(',')
    file.write(USD)
    file.write(',')
    
    file.write(markName)
    file.write(',')
    file.write(modelName)
    file.write(',')
    file.write(markNameEng)
    file.write(',')
    file.write(modelNameEng)
    file.write(',')
    file.write(raceInt)
    
    file.write(',')
    file.write(year)
    file.write(',')
    file.write(subCategoryNameEng)
    file.write(',')
    
    file.write(fuelName)
    file.write(',')
    file.write(fuelNameEng)
    file.write(',')
    file.write(gearboxName)
    file.write(',')
    
    file.write(custom)
    file.write(',')
    file.write(regionNameEng)
    file.write(',')
    file.write(locationCityName)
    file.write(',')
    file.write(count)
    file.write(',')
    file.write(userId)
    file.write(',')
    file.write(phone)
    
    
    file.write('\n')
    
    return _

### Get ids that were already processed

In [18]:
%%time
# read all ids and ids, that were already download
all_ids_file = open("data/already_retrieved.txt", "r")
ids_str = all_ids_file.read()
all_ids = ids_str.splitlines()
all_ids_file.close()

already_done_file = open("data/already_processed_ids.txt", "r")
ids_str = already_done_file.read()
already_done = set(ids_str.splitlines())
already_done_file.close()

ids_to_get = [x for x in all_ids if x not in already_done]


Wall time: 1.07 ms


In [19]:
len(ids_to_get)

490

## Get and write raw data + processed
### It's good to play around with error processing and track them frequently

In [20]:
already_done_file = open("data/already_processed_ids.txt", "a")
iter_n = 0
max_iter = 10#len(ids_to_get)
all_csv_file = open("data/all.csv", "a", encoding='utf-8')
errors_log_file = open("errors/errors.txt", "a")

try:
    while iter_n < max_iter:
        ido = get_random_id_and_del_from_list(ids_to_get)

        getstr = get_car_str + str(ido)
        print(getstr)
        resp = requests.get(getstr)

        print('request for car №' + str(iter_n) + ' was sent (id=' + str(ido) + ')')
        print(resp.status_code)

        if resp.status_code != 200:
            errors_log_file.write('\n')
            errors_log_file.write(str(ido) + ' - ' + str(resp.status_code))
            errors_log_file.write('\n')
            resp.close()
            time.sleep(3)
            continue
        else:
            try:
                json_info = resp.json()
                write_raw_info(ido, json_info, 'test/raw_json/')
                
                add_info_to_csv(ido, json_info, all_csv_file)
                already_done_file.write(ido)
                already_done_file.write('\n')
            except Exception as e:
                errors_log_file.write('\n')
                errors_log_file.write(str(ido) + ' - ' + str(type(e)) + ' - ' + str(e))
                errors_log_file.write('\n') 
            
            resp.close()

        iter_n += 1
finally:
    errors_log_file.flush()
    errors_log_file.close() 
    all_csv_file.flush()
    all_csv_file.close() 
    already_done_file.flush()
    already_done_file.close()  

489
https://developers.ria.com/auto/info?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&auto_id=22827235
request for car №0 was sent (id=22827235)
200
485
https://developers.ria.com/auto/info?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&auto_id=22951324
request for car №1 was sent (id=22951324)
200
351
https://developers.ria.com/auto/info?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&auto_id=22703708
request for car №2 was sent (id=22703708)
200
165
https://developers.ria.com/auto/info?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&auto_id=22782067
request for car №3 was sent (id=22782067)
200
442
https://developers.ria.com/auto/info?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&auto_id=22946790
request for car №4 was sent (id=22946790)
200
300
https://developers.ria.com/auto/info?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1lth5PfZ&auto_id=22738124
request for car №5 was sent (id=22738124)
200
287
https://developers.ria.com/auto/info?api_key=8rAuqdgyiS0ytCM3cOd06Xgyw79211tC1